In [11]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import timeseries_dataset_from_array
import matplotlib.pyplot as plt
import seaborn as sns
from MultiSeriesWindowsGenerator import MultiSeriesWindowsGenerator

In [12]:
pd.set_option("display.max_row", 40)

In [13]:
data_list = []
for i in range(0, 27):
    df = pd.read_csv(f"data/aggregated_individual_data_interpolation/interpolation/{i}_interpolated.csv", index_col=0)
    df["subject_id"] = i+1
    data_list.append(df)

# Concatenate the data into a single dataset
data = pd.concat(data_list)
data.drop(["circumplex.arousal_std","circumplex.valence_std","mood_std","activity_std"], inplace=True,axis=1)
# date_time = pd.to_datetime(df.pop('date'))
# df['days'] = date_time
# df = data

In [14]:
use_date = 0

if use_date:
    data.drop(["date"], inplace=True,axis=1)
    date_time = pd.to_datetime(data.pop('date'))
    df = data
    df['days'] = date_time

else: # use days from day 0 of recording
    data.drop(["date"], inplace=True,axis=1)
    df = data

df.reset_index(inplace=True,drop=True)
df = df.astype({'subject_id': 'float64','days': 'float64','weekday': 'float64'})

In [15]:
df = data
df.reset_index(inplace=True,drop=True)
df = df.astype({'subject_id': 'float64','days': 'float64','weekday': 'float64'})

In [16]:
LABELS = ['mood']
REGRESSORS = ['weekday', 'circumplex.arousal', 'circumplex.valence',
       'activity', 'screen', 'call', 'sms', 'appCat.builtin',
       'appCat.communication', 'appCat.entertainment', 'appCat.finance',
       'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
       'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

DATE = 'days' # always correct
IN_STEPS = 7 # use 7 days
OUT_STEPS = 7 # to predict 1 day in the future
GROUPBY = ['subject_id']
BATCH_SIZE = 8

In [17]:
n = len(df)
train_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(lambda x: x.iloc[:int(len(x) * 0.7)]).reset_index(drop=True)
val_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(lambda x: x.iloc[int(len(x) * 0.7):int(len(x) * 0.9)]).reset_index(drop=True)
test_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(lambda x: x.iloc[int(len(x) * 0.9):]).reset_index(drop=True)
train_series.shape, val_series.shape, test_series.shape

((804, 22), (234, 22), (130, 22))

# Baseline Model

In [18]:
cur_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=1, batch_size=BATCH_SIZE, GROUPBY=GROUPBY, label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)
cur_window.update_datasets(train_series, val_series, test_series, norm=True)

Error while processing dataset Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'mood'
Error while processing dataset Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'mood'
Error while processing dataset Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'mood'


In [19]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [20]:
baseline = Baseline(label_index=cur_window.column_indices[LABELS[0]])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(cur_window.val)
performance['Baseline'] = baseline.evaluate(cur_window.test, verbose=0)

ValueError: in user code:

    File "/Users/paulhosek/PycharmProjects/DataMining/MultiSeriesWindowsGenerator.py", line 124, in split_window  *
        inputs = features[:, self.input_slice, :]

    ValueError: Index out of range using input dim 2; input has only 2 dims for '{{node strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=5, ellipsis_mask=0, end_mask=5, new_axis_mask=0, shrink_axis_mask=0](args_0, strided_slice/stack, strided_slice/stack_1, strided_slice/stack_2)' with input shapes: [?,?], [3], [3], [3] and with computed input tensors: input[3] = <1 1 1>.


In [ ]:
def plot_2(self, model=None, plot_col=None, max_subplots=3):
    inputs, labels = self.example
    if not plot_col:
        plot_col = self.LABELS[0]
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))

    for n in range(max_n):
        plt.subplot(max_n, 1, n + 1)
        plt.ylabel(f'{plot_col} {"[normed]" if self.norm else ""}')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            print(self.label_indices, predictions[n, :, label_col_index])
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time [Days]')
    plt.show()

In [ ]:
cur_window.plot(baseline)

# RNN

#### define window


In [ ]:
LABELS = ['mood']
REGRESSORS = ['weekday', 'circumplex.arousal', 'circumplex.valence',
       'activity', 'screen', 'call', 'sms', 'appCat.builtin',
       'appCat.communication', 'appCat.entertainment', 'appCat.finance',
       'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
       'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

DATE = 'days' # always correct
IN_STEPS = 7 # use 7 days
OUT_STEPS = 7 # to predict 1 day in the future
GROUPBY = ['subject_id']
BATCH_SIZE = 7

In [ ]:
wide_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=OUT_STEPS, batch_size=BATCH_SIZE, GROUPBY = GROUPBY,
    label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)
wide_window.update_datasets(train_series, val_series, test_series, norm=True)
wide_window

#### training procedure

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', lstm_model(wide_window.example[0]).shape)

In [ ]:
import IPython.display

In [ ]:
wide_window.test_df

In [ ]:
history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
wide_window.plot(lstm_model)

# Autoregressive RNN